In [1]:
%pylab inline
from sklearn import ensemble, cross_validation, learning_curve, metrics 

import numpy as np
import pandas as pd

# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score

from tqdm import tqdm
import pickle
import seaborn as sns
# data review

train_df = pd.read_csv('train.csv',sep='\t')
test_df = pd.read_csv('test.csv',sep='\t')

Populating the interactive namespace from numpy and matplotlib


C:\Users\1\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\1\Anaconda2\lib\site-packages\sklearn\learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


In [2]:
X=train_df.drop(['0'], axis=1)
Y=train_df['0']
X=X.drop(['Unnamed: 0'], axis=1)

test_df = pd.read_csv('test.csv',sep='\t')
X_val=test_df.drop(['Unnamed: 0'], axis=1)
X_val=X_val.drop(['0'], axis=1)

In [3]:
X_concat = pd.concat([X, X_val]).reset_index(drop=True)
#//...
#X = X_concat.iloc[0:len(X)]
#X_tt = X_concat.iloc[len(X): ] 

In [4]:
cat_columns = []
for i in X_concat.columns: 
    if(len(X_concat[i].unique())<=30): 
        cat_columns.append(i)

X_concatC = pd.get_dummies(data=X_concat, columns=cat_columns)
X_concatC.head()

,13,66,67,68,69,70,71,72,73,74,...,341_0.888888889,341_1.0,342_0,342_1,343_0,343_1,344_0,344_1,345_0,345_1
0,0.461538,0.0,0.00000,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.0,...,0,0,0,1,0,1,0,1,0,1
1,0.187500,0.0,0.00000,0.0,0.0,0.0,0.46291,0.00000,0.15430,0.0,...,0,0,0,1,0,1,0,1,1,0
2,0.200000,0.0,0.00000,0.0,0.0,0.0,0.00000,0.73030,0.36515,0.0,...,0,0,0,1,0,1,0,1,0,1
3,0.000000,0.0,0.00000,0.0,0.0,0.0,0.36823,0.36823,0.00000,0.0,...,0,0,0,1,0,1,0,1,1,0
4,0.650000,0.0,0.07372,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.0,...,0,0,0,1,0,1,0,1,0,1


In [5]:
X_train1=X_concatC.iloc[0:len(X)]
X_pred = X_concatC.iloc[len(X): ] 

In [10]:
def pred_to_file(pred):
    pred = pd.DataFrame(pred, columns=["_VAL_"])
    pred.to_csv('solution_cat.csv', index_label="_ID_")

In [128]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict

class DjStacking(BaseEstimator, ClassifierMixin):  
    """Стэкинг моделей scikit-learn"""

    def __init__(self, models, ens_model):
        """
        Инициализация
        models - базовые модели для стекинга
        ens_model - мета-модель
        """
        self.models = models
        self.ens_model = ens_model
        self.n = len(models)
        self.valid = None
        
    def fit(self, X, y=None, p=0.25, cv=3, err=0.001, random_state=None):
        """
        Обучение стекинга
        p - в каком отношении делить на обучение / тест
            если p = 0 - используем всё обучение!
        cv  (при p=0) - сколько фолдов использовать
        err (при p=0) - величина случайной добавки к метапризнакам
        random_state - инициализация генератора
            
        """
        if (p > 0): # делим на обучение и тест
            # разбиение на обучение моделей и метамодели
            train, valid, y_train, y_valid = train_test_split(X, y, test_size=p, random_state=random_state)
            
            # заполнение матрицы для обучения метамодели
            self.valid = np.zeros((valid.shape[0], self.n))
            for t, clf in enumerate(self.models):
                clf.fit(train, y_train)
                self.valid[:, t] = clf.predict(valid)
                
            # обучение метамодели
            self.ens_model.fit(self.valid, y_valid)
            
        else: # используем всё обучение
            
            # для регуляризации - берём случайные добавки
            self.valid = err*np.random.randn(X.shape[0], self.n)
            
            for t, clf in enumerate(self.models):
                # это oob-ответы алгоритмов
                self.valid[:, t] += cross_val_predict(clf, X, y, cv=cv, n_jobs=-1, method='predict')
                # но сам алгоритм надо настроить
                clf.fit(X, y)
            
            # обучение метамодели
            self.ens_model.fit(self.valid, y)  
            

        return self
    


    def predict(self, X, y=None):
        """
        Работа стэкинга
        """
        # заполение матрицы для мета-классификатора
        X_meta = np.zeros((X.shape[0], self.n))
        
        for t, clf in enumerate(self.models):
            X_meta[:, t] = clf.predict(X)
        
        a = self.ens_model.predict(X_meta)
        
        return (a)

In [129]:
from mlxtend.classifier import StackingClassifier
from mlxtend.feature_selection import ColumnSelector
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.neural_network import MLPRegressor

In [130]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold 
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, RidgeCV, LassoCV, Ridge, Lasso

In [131]:
train_X, test_X, train_y, test_y = train_test_split(X_train1, Y, train_size=0.7, random_state=1999)

C:\Users\1\Anaconda2\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [132]:
train_X.head()

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [133]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor


knn1= MLPRegressor(solver='lbfgs', alpha=1e-2,hidden_layer_sizes=(4, 1), random_state=1,activation='logistic',learning_rate='invscaling',learning_rate_init=0.00001,verbose=1)
knn1.fit(train_X, train_y)
#run_and_plot(knn1, test_X, test_y, '3NN')

knn2 = KNeighborsRegressor(n_neighbors=10)
knn2.fit(train_X, train_y)
#run_and_plot(knn2, test_X, test_y, '10NN')


rg0 = Ridge(alpha=0.01)
rg0.fit(train_X, train_y)
#run_and_plot(rg0, test_X, test_y, 'ridge-0.01')

rg1 = Ridge(alpha=1.1)
rg1.fit(train_X, train_y)
#run_and_plot(rg1, test_X, test_y, 'ridge-1.1')

rg2 = Ridge(alpha=100.1)
rg2.fit(train_X, train_y)
#run_and_plot(rg2, test_X, test_y, 'ridge-100.1')


rf1 = RandomForestRegressor(n_estimators=1000, max_depth=1)
rf1.fit(train_X, train_y)
#run_and_plot(rf1, test_X, test_y, 'rf-d1')

rf2 = RandomForestRegressor(n_estimators=100, max_depth=5)
rf2.fit(train_X, train_y)
#run_and_plot(rf2, test_X, test_y, 'rf-d5')


gbm1 = lgb.LGBMRegressor(boosting_type='gbdt', learning_rate=0.05, max_depth=2, n_estimators=200, nthread=-1, objective='regression')    
gbm1.fit(train_X, train_y)
#run_and_plot(gbm1, test_X, test_y, 'gbm-d2')

gbm2 = lgb.LGBMRegressor(boosting_type='gbdt', learning_rate=0.05, max_depth=5, n_estimators=200, nthread=-1, objective='regression')    
gbm2.fit(train_X, train_y)
#run_and_plot(gbm2, test_X, test_y, 'gbm-d5')

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.05, max_depth=5, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=200,
       n_jobs=-1, nthread=-1, num_leaves=31, objective='regression',
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=1)

In [134]:
models = [knn1, knn2,rg1, rg2, rf1, rf2, gbm1, gbm2] # , rf3
ens_model = Ridge()
s1 = DjStacking(models, ens_model)
s1.fit(train_X, train_y)
roc_auc_score(test_y,s1.predict(test_X))

0.7196222514845007

In [135]:
s1.fit(X_train1, Y)  
roc_auc_score(Y, s1.predict(X_train1))

KeyboardInterrupt: 

In [16]:
preds=s1.predict(X_pred)

In [17]:
pred_to_file(preds)

# 0.749

# lets do ensemble of ensembles

In [22]:
train_X1=train_X[:(len(train_X))/2]
train_Y1=train_y[:(len(train_y))/2]

In [23]:
train_X2=train_X[(len(train_X))/2:]
train_Y2=train_y[(len(train_y))/2:]

In [24]:
knn11= MLPRegressor(solver='lbfgs', alpha=1e-2,hidden_layer_sizes=(4, 1), random_state=1,activation='logistic',learning_rate='invscaling',learning_rate_init=0.00001,verbose=1)
knn11.fit(train_X1, trainY1)
#run_and_plot(knn1, test_X, test_y, '3NN')

knn21 = KNeighborsRegressor(n_neighbors=10)
knn21.fit(train_X2, trainY2)
#run_and_plot(knn2, test_X, test_y, '10NN')


rg01 = Ridge(alpha=0.01)
rg01.fit(train_X1, trainY1)
#run_and_plot(rg0, test_X, test_y, 'ridge-0.01')

rg11 = Ridge(alpha=1.1)
rg11.fit(train_X2, trainY2)
#run_and_plot(rg1, test_X, test_y, 'ridge-1.1')

rg21 = Ridge(alpha=100.1)
rg21.fit(train_X1, trainY1)
#run_and_plot(rg2, test_X, test_y, 'ridge-100.1')


rf11 = RandomForestRegressor(n_estimators=1000, max_depth=1)
rf11.fit(train_X1, trainY1)
#run_and_plot(rf1, test_X, test_y, 'rf-d1')

rf21 = RandomForestRegressor(n_estimators=100, max_depth=5)
rf21.fit(train_X2, trainY2)
#run_and_plot(rf2, test_X, test_y, 'rf-d5')


gbm11 = lgb.LGBMRegressor(boosting_type='gbdt', learning_rate=0.05, max_depth=2, n_estimators=200, nthread=-1, objective='regression')    
gbm11.fit(train_X1, trainY1)
#run_and_plot(gbm1, test_X, test_y, 'gbm-d2')

gbm21 = lgb.LGBMRegressor(boosting_type='gbdt', learning_rate=0.05, max_depth=5, n_estimators=200, nthread=-1, objective='regression')    
gbm21.fit(train_X2, trainY2)
#run_and_plot(gbm2, test_X, test_y, 'gbm-d5')

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.05, max_depth=5, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=200,
       n_jobs=-1, nthread=-1, num_leaves=31, objective='regression',
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=1)

In [28]:
models1 = [knn11, knn21,rg11, rg21, rf11, rf21,gbm11, gbm21] # , rf3
ens_model1 = Ridge()
s1 = DjStacking(models1, ens_model1)
s1.fit(train_X, train_y)
roc_auc_score(train_y,s1.predict(train_X))

0.8136921721886591

In [30]:
s1.fit(X_train1, Y)  
roc_auc_score(Y, s1.predict(X_train1))

0.813533034830263

In [38]:
preds=s1.predict(X_pred)

In [39]:
pred_to_file(preds)

# cat features combining

In [117]:
cat_columns1 = []
for i in X_concatC.columns: 
    if(len(X_concatC[i].unique())<=2): 
        cat_columns1.append(i)

In [118]:
for j in cat_columns1:
    for j in cat_columns1:
        if i!=j:
            if i.split('_')[0] != j.split('_')[0] :
                X_concatC[i+j]=X_concatC[i].T*X_concatC[j]

In [119]:
X_concatC=X_concatC.drop(cat_columns1,axis=1)

In [120]:
for i in X_concatC.columns:
    if len(X_concatC[i].unique()) == 1:
        X_concatC=X_concatC.drop(i,axis=1)

In [121]:
X_concatC.head()

,13,66,67,68,69,70,71,72,73,74,...,345_1341_0.666666667,345_1341_0.777777778,345_1341_0.888888889,345_1341_1.0,345_1342_0,345_1342_1,345_1343_0,345_1343_1,345_1344_0,345_1344_1
0,0.461538,0.0,0.00000,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.0,...,0,0,0,0,0,1,0,1,0,1
1,0.187500,0.0,0.00000,0.0,0.0,0.0,0.46291,0.00000,0.15430,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0.200000,0.0,0.00000,0.0,0.0,0.0,0.00000,0.73030,0.36515,0.0,...,0,0,0,0,0,1,0,1,0,1
3,0.000000,0.0,0.00000,0.0,0.0,0.0,0.36823,0.36823,0.00000,0.0,...,0,0,0,0,0,0,0,0,0,0
4,0.650000,0.0,0.07372,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.0,...,0,0,0,0,0,1,0,1,0,1


In [61]:
X_train1=X_concatC.iloc[0:len(X)]
X_pred = X_concatC.iloc[len(X): ] 

In [62]:
X_train1.shape

(30500, 763)

In [104]:
X_pred.shape

(4166, 763)

 # PCA
 

In [6]:
from sklearn.decomposition import PCA

In [119]:
model = PCA(n_components=180,random_state=1)
model.fit(X_train1)

PCA(copy=True, iterated_power='auto', n_components=180, random_state=1,
  svd_solver='auto', tol=0.0, whiten=False)

In [120]:
X_new=model.transform(X_train1)
X_pred_new=model.transform(X_pred)

In [121]:
from sklearn.neural_network import MLPRegressor

In [122]:
clf = MLPRegressor(solver='lbfgs', alpha=1e-2,hidden_layer_sizes=(40, 2), random_state=1,activation='tanh',learning_rate='invscaling',learning_rate_init=0.00001,verbose=1)
clf.fit(X_new, Y)  
roc_auc_score(Y,clf.predict(X_new))

0.8333625739011876

120 comp - 40,2 -0.810 
140 comp - 40,2 -0.81734
160 comp - 40,2 -0.830
180 comp - 40,2 -0.833
200 comp - 40,2 -0.825
180 comp is best

In [123]:
(X_train, 
 X_test, 
 y_train, y_test) = train_test_split(X_new, Y, 
                                     test_size=0.3, 
                                     random_state=1)


In [127]:
clf = MLPRegressor(solver='lbfgs', alpha=1e-2,hidden_layer_sizes=(5, 1), random_state=1,activation='tanh',learning_rate='invscaling',learning_rate_init=0.00001,verbose=1)
clf.fit(X_train, y_train)  
roc_auc_score(y_test,clf.predict(X_test))

0.709689424993611